In [1]:
import numpy as np
import numpy.random as rnd
import scipy
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.optimize import minimize
from numpy import linalg as LA
import cvxpy as cp
import matplotlib.pyplot as plt

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [11]:
from exp_setup import Experiment

In [3]:
from algorithms import *

In [4]:
num_species = 4 #drone,rover,mini-rover,mini-drone
num_tasks = 3  #pick,search for target,move object
num_traits = 9 #speed,footprint,payload,reach,weight,sensing frequency,sensing range,color,battery capacity
traits = ["speed","footprint","payload","reach","weight","sensing frequency","sensing range","color","battery"]
num_demo = 1000

In [5]:
exp = Experiment()
D = exp.get_expert_demonstrations()

In [21]:
new_Q = exp.get_random_q()
agents_target = exp.get_agent_limit()
Q = D["Q"]
Y_mean = np.mean(D["Y"], axis=0)
new_Q

array([[  9.23459262,  87.31264947,   8.61065748,   1.15583504,
         26.85296609, 214.64485234,  58.36983768,   2.        ,
         18.        ],
       [  4.59852235, 135.25506881,   3.31303991,   2.25062842,
         26.63073849, 196.09072662,  58.21801235,   0.        ,
         16.        ],
       [  6.9029882 , 118.8952442 ,  15.51899725,   1.96853678,
         27.6905935 , 255.1398822 ,  58.90387257,   1.        ,
         16.        ],
       [ 12.44834684,  79.75577512,   9.02910972,   0.56634217,
         26.41638599, 224.76446829,  58.68997651,   0.        ,
         19.        ]])

In [22]:
X_mat = solve_task_allocation(agents_target,new_Q,Y_mean)
print("mat" , X_mat)

mat [[ 0.  0.  1. 10.]
 [ 5.  4.  4.  0.]
 [ 0.  4.  0.  0.]]


In [23]:
X_per_task = solve_task_allocation_per_task(agents_target,new_Q,Y_mean)
print("task based" , X_per_task)

task based [array([ 0.,  0.,  1., 10.]), array([5., 4., 4., 0.]), array([0., 4., 0., 0.])]


In [24]:
Y_mat = X_mat@new_Q
print(Y_mat)

Y_per_task = X_per_task@new_Q
print(Y_per_task)

[[1.31386457e+02 9.16452995e+02 1.05810094e+02 7.63195844e+00
  2.91854453e+02 2.50278457e+03 6.45803638e+02 1.00000000e+00
  2.06000000e+02]
 [9.21790053e+01 1.45316450e+03 1.18381436e+02 2.26558360e+01
  3.51550158e+02 2.87814670e+03 7.60336728e+02 1.40000000e+01
  2.18000000e+02]
 [1.83940894e+01 5.41020275e+02 1.32521596e+01 9.00251367e+00
  1.06522954e+02 7.84362906e+02 2.32872049e+02 0.00000000e+00
  6.40000000e+01]]
[[1.31386457e+02 9.16452995e+02 1.05810094e+02 7.63195844e+00
  2.91854453e+02 2.50278457e+03 6.45803638e+02 1.00000000e+00
  2.06000000e+02]
 [9.21790053e+01 1.45316450e+03 1.18381436e+02 2.26558360e+01
  3.51550158e+02 2.87814670e+03 7.60336728e+02 1.40000000e+01
  2.18000000e+02]
 [1.83940894e+01 5.41020275e+02 1.32521596e+01 9.00251367e+00
  1.06522954e+02 7.84362906e+02 2.32872049e+02 0.00000000e+00
  6.40000000e+01]]


In [25]:
optimal_Y = np.array([[ 122.53839519, 0., 119.39044525, 32.15983648, 360.87624558, 2920.25543052, 768.93055775, 0., 0.],
                                   [ 120.64147124, 1433.4094241 ,  0.,   0., 358.13547136, 2906.93229221,  766.11629311,   0., 190.],
                                   [ 122.31492305, 1441.74082593,  119.06910271,   0., 358.78852644, 0. ,  769.11520065,   0., 193.]])


In [26]:
LA.norm(optimal_Y-Y_per_task, 2)/LA.norm(optimal_Y,2)

0.36131178575726414

In [27]:
LA.norm(optimal_Y-Y_mat, 2)/LA.norm(optimal_Y,2)

0.36131178575726414